<a href="https://colab.research.google.com/github/LoniQin/LLM-resources/blob/main/Chatbot_with_mamba_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot with mamba-chat

1. First download and install repository https://github.com/havenhq/mamba-chat.git. If you are in Google Colab, you may run following command to avoid issue:
```
!echo /usr/lib64-nvidia/ >/etc/ld.so.conf.d/libcuda.conf; ldconfig
```

In [1]:
!git clone https://github.com/havenhq/mamba-chat.git
!pip install -r mamba-chat/requirements.txt
!echo /usr/lib64-nvidia/ >/etc/ld.so.conf.d/libcuda.conf; ldconfig

Cloning into 'mamba-chat'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 113 (delta 57), reused 46 (delta 46), pack-reused 52
Receiving objects: 100% (113/113), 730.12 KiB | 14.04 MiB/s, done.
Resolving deltas: 100% (58/58), done.


Open terminal, go to the downloaded github repository and input following command. Finally you can chat with Mamba in terminal.
```
python chat.py
```

Now I will download the model which is same with mamba-chat to have a conversation with.

In [4]:
import torch
import gc
from transformers import AutoTokenizer
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

## Load Mamba model

In [5]:
device = "cuda"
model = MambaLMHeadModel.from_pretrained("havenhq/mamba-chat", device=device, dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("havenhq/mamba-chat")
eos = "<|endoftext|>"
tokenizer.eos_token = eos
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = AutoTokenizer.from_pretrained(
  "HuggingFaceH4/zephyr-7b-beta"
).chat_template

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
def chat_with_mamba(
        user_message,
        history: list[list[str]],
        temperature: float = 0.9,
        top_p: float = 0.7,
        max_length: int = 2000,
):
        history_dict: list[dict[str, str]] = []
        for user_m, assistant_m in history:
            history_dict.append(dict(role="user", content=user_m))
            history_dict.append(dict(role="assistant", content=assistant_m))
        history_dict.append(dict(role="user", content=user_message))

        input_ids = tokenizer.apply_chat_template(
            history_dict, return_tensors="pt", add_generation_prompt=True
        ).to(device)

        out = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            eos_token_id=tokenizer.eos_token_id,
        )

        decoded = tokenizer.batch_decode(out)
        assistant_message = (
            decoded[0].split("<|assistant|>\n")[-1].replace(eos, "")
        )
        return assistant_message

In [19]:
conversations = []
def chat(question):
  answer = chat_with_mamba(question, conversations)
  conversations.append((question, answer))
  print(answer)

In [20]:
chat("Hi, John. How are you?")

I am doing well, thank you for asking. How about you?


In [21]:
chat("I feel really unhappy.")

I am sorry to hear that. Can you please tell me more about your unhappiness?


In [22]:
chat("New year is comming. I don't know how to make a plan for it.")

I can understand your frustration. It's normal to feel overwhelmed and uncertain about what to do during the new year. However, there are many things you can do to make your new year more successful. Here are some tips:

1. Set SMART goals: SMART goals are specific, measurable, achievable, realistic, and time-bound. They help you set clear and achievable goals for the new year.

2. Set a budget: Make a budget for your new year's goals. This will help you stay within your limits and avoid overspending.

3. Set reminders: Set reminders on your phone or calendar to remind you of your goals and tasks.

4. Set aside time: Set aside a specific time each day to work on your goals. This will help you stay focused and avoid distractions.

5. Seek support: Seek support from friends, family, or a coach to help you stay motivated and focused.

6. Celebrate your successes: Celebrate your successes and acknowledge your progress. This will help you stay motivated and keep you motivated.

Remember, th

In [23]:
chat("Thank you very much. I think I am more eager to lose weight. Could you give some suggestions?")

Sure, here are some suggestions to help you lose weight:

1. Set a realistic goal: Set a realistic goal for your weight loss. Don't aim for unrealistic weight loss goals that are too difficult to achieve. Instead, aim for a healthy weight loss of 1-2 pounds per week.

2. Track your progress: Track your progress regularly by using a weight loss app or a food diary. This will help you stay motivated and track your progress.

3. Eat healthy: Eat a balanced diet that includes lean proteins, fruits, vegetables, and whole grains. Avoid processed foods and sugar.

4. Exercise regularly: Exercise regularly to burn calories and boost your metabolism. Choose activities that you enjoy and that you can sustain for a long time.

5. Stay hydrated: Drink plenty of water throughout the day to keep your body hydrated and to help you feel full.

6. Avoid sugar: Avoid sugar and processed foods that are high in sugar. Instead, choose whole foods that are low in sugar.

Remember, weight loss is a journey, 